# A100

In [1]:
import os

format = """#!/bin/bash
#SBATCH --job-name={method}_HSIC_{dataset}
#SBATCH --partition=kisski
#SBATCH --gres=gpu:A100:1
#SBATCH --cpus-per-task=16
#SBATCH --mem=20G
#SBATCH --time=2-00:00:00
#SBATCH --output=run_%x_%j.out
#SBATCH --error=run_%x_%j.err

#SBATCH --mail-type=FAIL          # Email on start, end, failure
#SBATCH --mail-user=hao.huang@tib.eu   # <-- Replace with your real email


echo "===== JOB STARTED ====="
echo "Hostname: $(hostname)"
echo "Date:     $(date)"
echo "User:     $USER"


# Load environment
module load miniforge3
module load gcc/13.2.0
module load cuda/11.8.0
# module load gcc/13.2.0
# module load cuda/12.6.2

# Set up conda
source "$(conda info --base)/etc/profile.d/conda.sh"
conda activate /mnt/vast-kisski/projects/kisski-tib-activecl/cenv

# Diagnostics
echo "Which python: $(which python)"
python -c "import torch; print('Torch:', torch.__version__, '| CUDA:', torch.cuda.is_available())"

# Navigate to project directory
cd ~/GCN_policy_ma

# Run scripts
echo "--- Running: main_other_methods.py {dataset} ---"
python3 -u train_policy.py --dataset {dataset} --missing_p 0.0 --gnn_fun {method} --ablation {ablation}
python3 -u train_policy.py --dataset {dataset} --missing_p 0.1 --gnn_fun {method} --ablation {ablation}
python3 -u train_policy.py --dataset {dataset} --missing_p 0.3 --gnn_fun {method} --ablation {ablation}

echo "===== JOB COMPLETED ====="
echo "Date: $(date)"
"""

datasets = ['Syn_M=None_SimRel=1_Rel=4',  'BlogCatalog1_M=20_SimRel=1_Rel=4', 'Flickr1_M=20_SimRel=1_Rel=4', 'Youtube_M=20_SimRel=1_Rel=4']
slurm_dir = 'slurm_a100_ablation'
methods = ['GCN'] #, "GraphSAGE"]
ablations = ['no', 'no_rep', '1y_fun']
dropout = [0.1, 0.5]

# Create the slurm directory if it doesn't exist
os.makedirs(slurm_dir, exist_ok=True)

# print(f"Generating SLURM files in '{slurm_dir}' and printing sbatch commands:\n")

# Generate and save the slurm files and print commands
print("cd GCN_policy_ma")
for dataset in datasets:
    for method in methods:
        for ablation in ablations:
            # Fill the format string with the current dataset name
            slurm_content = format.format(dataset=dataset, method=method, ablation=ablation)

            # Define the output filename
            filename = os.path.join(slurm_dir, f"{method}_{dataset}_{ablation}.slurm")

            # Write the content to the file
            with open(filename, "w") as f:
                f.write(slurm_content)

            # print(f"Generated {filename}")

            sbatch_command = f"sbatch {filename}"
            print(sbatch_command)


cd GCN_policy_ma
sbatch slurm_a100_ablation/GCN_Syn_M=None_SimRel=1_Rel=4_no.slurm
sbatch slurm_a100_ablation/GCN_Syn_M=None_SimRel=1_Rel=4_no_rep.slurm
sbatch slurm_a100_ablation/GCN_Syn_M=None_SimRel=1_Rel=4_1y_fun.slurm
sbatch slurm_a100_ablation/GCN_BlogCatalog1_M=20_SimRel=1_Rel=4_no.slurm
sbatch slurm_a100_ablation/GCN_BlogCatalog1_M=20_SimRel=1_Rel=4_no_rep.slurm
sbatch slurm_a100_ablation/GCN_BlogCatalog1_M=20_SimRel=1_Rel=4_1y_fun.slurm
sbatch slurm_a100_ablation/GCN_Flickr1_M=20_SimRel=1_Rel=4_no.slurm
sbatch slurm_a100_ablation/GCN_Flickr1_M=20_SimRel=1_Rel=4_no_rep.slurm
sbatch slurm_a100_ablation/GCN_Flickr1_M=20_SimRel=1_Rel=4_1y_fun.slurm
sbatch slurm_a100_ablation/GCN_Youtube_M=20_SimRel=1_Rel=4_no.slurm
sbatch slurm_a100_ablation/GCN_Youtube_M=20_SimRel=1_Rel=4_no_rep.slurm
sbatch slurm_a100_ablation/GCN_Youtube_M=20_SimRel=1_Rel=4_1y_fun.slurm


# CPU